In [2]:
import pybiopax
from collections import Counter
import pprint

In [3]:
biopax_file = './resources/test_biopax/R-HSA-204174_level3.owl'
model = pybiopax.model_from_owl_file(biopax_file, encoding="utf8")

Processing OWL elements: 100%|██████████████████████████████████████████████████| 423/423 [00:00<00:00, 9.20kit/s]


In [4]:
stats = Counter([obj.__class__.__name__  for uid, obj in model.objects.items()]).most_common(100)
stats

[('UnificationXref', 160),
 ('Stoichiometry', 37),
 ('SequenceSite', 34),
 ('RelationshipXref', 25),
 ('PublicationXref', 24),
 ('Protein', 18),
 ('ProteinReference', 16),
 ('FragmentFeature', 16),
 ('SequenceInterval', 16),
 ('SmallMolecule', 15),
 ('SmallMoleculeReference', 15),
 ('Complex', 15),
 ('Control', 8),
 ('BiochemicalReaction', 4),
 ('RelationshipTypeVocabulary', 4),
 ('PathwayStep', 4),
 ('Catalysis', 3),
 ('CellularLocationVocabulary', 2),
 ('ModificationFeature', 2),
 ('Pathway', 1),
 ('Provenance', 1),
 ('BioSource', 1),
 ('SequenceModificationVocabulary', 1)]

In [ ]:
for protein in model.get_objects_by_type(pybiopax.biopax.Pathway):
  
    pprint.pp(dir(protein))
    break
   

In [9]:
for pathway in model.get_objects_by_type(pybiopax.biopax.Pathway):
    name = pathway
    print(name.uid)
    for obj in pathway.pathway_order:
        pprint.pp('%s -> %s' % ('Pathway Order', obj.uid))
        
        pc = model.objects[obj.uid]   
        #pprint.pp(pc.__dict__)
        for sp in pc.step_process:            
            if isinstance (sp, pybiopax.biopax.Catalysis) and sp.control_type == 'ACTIVATION':
                catalysis = model.objects[sp.uid]     
                for xref in sp.xref:
                    if(xref.db == 'GENE ONTOLOGY'):
                        print('\t %s --> %s' % (xref.uid, xref.id))
        
    for obj in pathway.pathway_component:
        pprint.pp('%s -> %s' % (obj.uid, obj.display_name))
        pc = model.objects[obj.uid]
                   
        for l in pc.left:
            print('\t %s -> %s' % ('has_input',  l.display_name))
            
        for r in pc.right:
            print('\t %s --> %s' % ('has_output',  r.display_name))
        
   

Pathway1
'Pathway Order -> PathwayStep1'
	 RelationshipXref4 --> GO:0019120
'Pathway Order -> PathwayStep2'
'Pathway Order -> PathwayStep3'
	 RelationshipXref15 --> GO:0004740
'Pathway Order -> PathwayStep4'
	 RelationshipXref22 --> GO:0004741
'BiochemicalReaction1 -> GSTZ1 dimer dehalogenates DCA to glyoxylate'
	 has_input -> H2O
	 has_input -> DCA
	 has_output --> glyoxylate
	 has_output --> HCl
'BiochemicalReaction2 -> DCA binds PDK2'
	 has_input -> PDK2
	 has_input -> DCA
	 has_output --> DCA:PDK2
'BiochemicalReaction3 -> PDK isoforms phosphorylate lipo-PDH'
	 has_input -> ATP
	 has_input -> lipo-PDH
	 has_output --> ADP
	 has_output --> p-lipo-PDH
'BiochemicalReaction4 -> PDP dephosphorylates p-lipo-PDH'
	 has_input -> p-lipo-PDH
	 has_input -> H2O
	 has_output --> lipo-PDH
	 has_output --> Pi
